In [ ]:
import numpy as np
import torch
import math 
from torch.utils.data import Dataset


def calculate_fire_spread_rate(wind_speed_at_10m, moisture_content, slope_angle, fuel_load=2):
    """
    Calculate the rate of spread of a forest fire.

    Parameters:
    wind_speed_at_10m (float): Wind speed at 10 meters height in km/hr.
    moisture_content (float): Moisture content in percentage (35% - 65%).
    slope_angle (float): Slope angle in degrees.
    fuel_load (float): Fuel load in tonnes/hectare (< 6 mm diameter), default is 2 tonnes/hectare.

    Returns:
    float: Rate of spread of the fire in unspecified units.
    """
    # Constants
    a = 1.674
    b = 0.1798
    c = 0.22
    d = 0.158
    e = -0.227
    f = 0.0662

    # Convert wind speed at 10m to wind speed at 1.5m
    wind_speed_at_1_5m = a + b * wind_speed_at_10m

    # Calculate the rate of spread on flat ground
    rate_flat_ground = c * fuel_load * math.exp((wind_speed_at_1_5m ** d) * (moisture_content ** e))

    # Adjust rate for slope
    rate_on_slope = rate_flat_ground * math.exp((slope_angle ** f))

    return rate_flat_ground 

# Example usage
spread_rate = calculate_fire_spread_rate(20, 50, 10, 2)
spread_rate

In [ ]:
from scipy.stats import qmc
import pandas as pd


# Define the bounds for each parameter
bounds = np.array([[11.5, 34.1],  # Wind speed
                   [0, 6],        # Fuel load
                   [35, 65],      # Moisture content
                   [0, 30]])      # Slope angle

# Using Latin Hypercube Sampling (LHS) for a more structured approach
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=150000)
sample = qmc.scale(sample, bounds[:, 0], bounds[:, 1])

# Extracting the samples for each parameter
wind_speed_samples = sample[:, 0]
fuel_samples = sample[:, 1]
moisture_content_samples = sample[:, 2]
slope_angle_samples = sample[:, 3]

# Calculating spread rates for the sampled data
spread_rates = [calculate_fire_spread_rate(wind_speed, moisture, slope, fuel) 
                for wind_speed, moisture, slope, fuel in zip(wind_speed_samples, moisture_content_samples, slope_angle_samples, fuel_samples)]

# Creating a DataFrame to hold the sampled data and calculated spread rates
data_lhs = pd.DataFrame({
    'Wind Speed (km/hr)': wind_speed_samples,
    'Fuel Load (tonnes/hectare)': fuel_samples,
    'Moisture Content (%)': moisture_content_samples,
    'Slope Angle (degrees)': slope_angle_samples,
    'Spread Rate': spread_rates
})

data_lhs.head()

In [ ]:

import torch
from torch.utils.data import Dataset

class FireSpreadDataset(Dataset):
    def __init__(self, dataframe):
        # Convert the dataframe to a PyTorch tensor
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming the last column is the target variable (spread rate)
        features = self.data[idx, :-1]
        target = self.data[idx, -1]
        return features, target

# Convert the DataFrame to a PyTorch Dataset
fire_spread_data = FireSpreadDataset(data_lhs)

# Example: accessing the first item in the dataset
first_sample_features, first_sample_target = fire_spread_data[0]
print(first_sample_features, first_sample_target)


In [ ]:
print(fire_spread_data[0])

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from typing import List 
import torch.nn as nn
import torch.optim as optim

class BushfireModel(nn.Module): 
    def __init__(self, n: int, hidden_layers: List[int]): 
        super().__init__() #Inherit from the nn.module
        self.n = n # n input features
        self.hidden_layers = hidden_layers # hidden layers are the middle ones

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *hidden_layers], [*hidden_layers, 1])
        ]) # Single output

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:    #We don't want to ReLU the last layer
                x = nn.functional.relu(x)  
        return x # Linear output for regression

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Splitting the dataset
total_size = len(fire_spread_data)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation
train_dataset, val_dataset = random_split(fire_spread_data, [train_size, val_size])

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Training mode
        for i, (features, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(features)
            loss = loss_criterion(outputs, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # Validation mode
        with torch.no_grad():
            val_loss = 0
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += loss_criterion(outputs, labels.unsqueeze(1)).item()

        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Initialize the model
model = BushfireModel(n=4, hidden_layers=[700,300])
loss_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
num_epochs = 4
train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs)


In [75]:
def predict_spread_rate(model, wind_speed, moisture_content, fuel_load, slope_angle):
    # Convert input data to tensor
    input_tensor = torch.tensor([wind_speed, moisture_content, fuel_load, slope_angle], dtype=torch.float32)

    # Reshape input_tensor to match the input shape of the model
    input_tensor = input_tensor.unsqueeze(0)  # Adds a batch dimension

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        prediction = model(input_tensor)

    # Return the predicted spread rate
    return prediction.item()

# Example usage
predicted_spread_rate = predict_spread_rate(model, 30, 39, 0, 4)
print("Predicted Spread Rate:", predicted_spread_rate)

# Example usage
spread_rate = calculate_fire_spread_rate(30, 39, 0, 4)
print("Actual Spread Rate:", spread_rate)


Predicted Spread Rate: 6.393197536468506
Actual Spread Rate: 1.5921979484515598
